In [3]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count =numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print ' '*ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind+1)

In [4]:
rootNode = treeNode('pyramid',9, None)

In [5]:
rootNode.children['eye'] = treeNode('eye', 13, None)

In [6]:
rootNode.disp()

  pyramid   9
   eye   13


In [7]:
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)

In [9]:
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


In [41]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    headerTableCopy = headerTable.copy()
    for k in headerTableCopy.keys():  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [42]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
              ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
              ['z'],
              ['r', 'x', 'n', 'o', 's'],
              ['y', 'r', 'x', 'z', 'q', 't', 'p'],
              ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [43]:
simpDat = loadSimpDat()

In [44]:
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [45]:
initSet = createInitSet(simpDat)

In [46]:
initSet

{frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1}

In [49]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [51]:
myFPtree.disp()

  Null Set   1
   x   1
    s   1
     r   1
   z   5
    x   3
     y   3
      s   2
       t   2
      r   1
       t   1
    r   1


In [52]:
myHeaderTab

{'r': [3, <__main__.treeNode instance at 0x0000000004D060C8>],
 's': [3, <__main__.treeNode instance at 0x0000000004CFB208>],
 't': [3, <__main__.treeNode instance at 0x0000000004CFBFC8>],
 'x': [4, <__main__.treeNode instance at 0x0000000004CFB088>],
 'y': [3, <__main__.treeNode instance at 0x0000000004CFB1C8>],
 'z': [5, <__main__.treeNode instance at 0x0000000004CFB048>]}

In [57]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [58]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [61]:
findPrefixPath('z', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [62]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [63]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'s', 'x'}): 1, frozenset({'z'}): 1, frozenset({'x', 'y', 'z'}): 1}

In [74]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [75]:
freqItems = []

In [76]:
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

('conditional tree for: ', set(['s']))
  Null Set   1
   x   3
('conditional tree for: ', set(['t']))
  Null Set   1
   y   3
    x   3
     z   3
('conditional tree for: ', set(['x', 't']))
  Null Set   1
   y   3
('conditional tree for: ', set(['z', 't']))
  Null Set   1
   y   3
    x   3
('conditional tree for: ', set(['x', 'z', 't']))
  Null Set   1
   y   3
('conditional tree for: ', set(['x']))
  Null Set   1
   z   3
('conditional tree for: ', set(['y']))
  Null Set   1
   x   3
    z   3
('conditional tree for: ', set(['y', 'z']))
  Null Set   1
   x   3


In [77]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'z'}]

In [82]:
import twitter
from time import sleep
import re

In [100]:
def textParse(bigString):
    urlsRemoved = re.sub('(http:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/.]|[~])*', '', bigString)    
    listOfTokens = re.split(r'\W*', urlsRemoved)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def getLotsOfTweets(searchStr):
    CONSUMER_KEY = '5EXkQJqgGkwmh86gcEf2wUKYM'
    CONSUMER_SECRET = '9d9a8VCE5YN0Rhky1hj25yQolQe1EIitJ290rDjzDSrf6LXUvB'
    ACCESS_TOKEN_KEY = '174599088-vNGKbOGHb8DehXuaMXebmvwq9wtHxbE1vEzyrw9X'
    ACCESS_TOKEN_SECRET = 'DzmD3UX7o6ZEPypJAistCkkD88BVhOqBL58UkE1x3PrPr'
    api = twitter.api(consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
                      access_token_key=ACCESS_TOKEN_KEY, 
                      access_token_secret=ACCESS_TOKEN_SECRET)
    #you can get 1500 results 15 pages * 100 per page
    resultsPages = []
    for i in range(1,15):
        print("fetching page %d" % i)
        searchResults = api.GetSearch(searchStr, per_page=100, page=i)
        resultsPages.append(searchResults)
        sleep(6)
    return resultsPages

def mineTweets(tweetArr, minSup=5):
    parsedList = []
    for i in range(14):
        for j in range(100):
            parsedList.append(textParse(tweetArr[i][j].text))
    initSet = createInitSet(parsedList)
    myFPtree, myHeaderTab = createTree(initSet, minSup)
    myFreqList = []
    mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)
    return myFreqList

In [102]:
#lotsOftweets = getLotsOfTweets('#')

In [103]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [104]:
initSet = createInitSet(parsedDat)

In [106]:
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [107]:
myFreqList = []

In [108]:
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

('conditional tree for: ', set(['1']))
  Null Set   1
   6   107404
('conditional tree for: ', set(['11']))
  Null Set   1
   6   261773
('conditional tree for: ', set(['3']))
  Null Set   1
   11   9718
   6   186289
    11   117401
('conditional tree for: ', set(['11', '3']))
  Null Set   1
   6   117401


In [109]:
len(myFreqList)

9

In [110]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]